
<center> <h1>Consiguiendo los datos con Tweepy</h1> </center>
   
   
![](img/1.jpg)

<h3>Juan Cuenca</h3>
[Twitter](http://twitter.com/juan_qenk) | [Github](https://github.com/juanqenk) | [mail](mailto:juanqenk@gmail.com)

**Tweepy** es una libreria escrita en `Python` para manejar la red social `Twitter` de una forma sencilla y directa. Con [Tweepy](http://www.tweepy.org/) podeis acceder a la `API` de Twitter y realizar busquedas, actualizar vuestro perfil o analizar a vuestros seguidores.

Tweepy es relativamente intuitiva y de fácil uso. Lo primero que necesitamos es instalar la librería en python, que como la mayoría de librerías puedes instalar con `pip` o `easy_install`. Abrid vuestra consola de comandos e instalad Tweepy con alguna de las siguientes órdenes:

**Instalación con pip:**
```bash
pip search tweepy # para buscar si esta en los repositorios
pip install tweepy
```
**Instalación con easy_install:**
```bash  
easy_install tweepy
```
Tened en cuenta que necesitais registrar vuestra aplicación en Twitter para comunicaros con la API, concretamente en la web de [Twitter para desarrolladores](https://apps.twitter.com/). Una vez os habeis registrado con vuestra cuenta y elegido los permisos correspondientes para la app, en `Keys and Access Tokens` podeis encontrar dos claves; API_key y API secret (en adelante `consumer_key` y `consumer_secret`, respectivamente). Necesitareis además generar los tokens; `access_token` y `access_secret`. Estas cuatro claves son necesarias para comunicaros con la red social. 

Con tal de no tener que copiar las claves cada vez que intenteis hacer una busqueda, os recomiendo guardarlas en un archivo vacio llamado `passw.py`. El contenido de este debe ser solo vuestras claves siguiendo este formato:
```python
consumer_key = 'sustituid el contenido de estas comillas por vuestro consumer key'
consumer_secret = 'sustituid el contenido de estas comillas por vuestro consumer secret'
access_token = 'sustituid el contenido de estas comillas por vuestro access token'
access_secret = 'sustituid el contenido de estas comillas por vuestro access secret'
```
Una vez hemos instalado Tweepy y generado el archivo con las contraseñas (el cual guardaremos en la carpeta de trabajo) podemos importarlos a nuestro script para su uso. 

In [2]:
import tweepy
from passw import consumer_key, consumer_secret, access_token, access_secret
import numpy as np
import pandas as pd
import platform
import sys

In [4]:
print("Sistema operativo:     ", platform.platform())
print("Versión de Python:     ", sys.version)
print("Versión de Pandas:     ", pd.__version__)
print("Versión de Numpy:      ", np.version.short_version)
print("Versión de Tweepy:     ", tweepy.__version__)

Sistema operativo:      Linux-4.9.71-1-MANJARO-x86_64-with-arch-Manjaro-Linux
Versión de Python:      3.6.1 |Continuum Analytics, Inc.| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Versión de Pandas:      0.21.0
Versión de Numpy:       1.13.3
Versión de Tweepy:      3.5.0


Ejecutando el código que se encuentra a continuación nos comunicaremos con Twitter para mandarle nuestras credenciales. Twitter comprobará que hemos creado correctamente una aplicación y obtendremos una autorización para conectarnos con la API. Finalmente obtenemos un objeto llamado `api` sobre el cual podemos ejecutar las diferentes funciones que nos ofrece Tweepy. 

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

Una vez hemos accedido, hay muchas cosas que podemos hacer:
- enviar un tuit
- seguir o dejar de seguir a otros
- actualizar el perfil
- obtener información de un usuario
- obtener todos los tuits de un usuario
- ver todos los seguidores de otro usuario
- hacer búsquedas de tuits
- guardar todos los tuits que van apareciendo en directo
- ...   

A modo de demostración, podemos obtener de manera sencilla la siguiente información:

In [6]:
usuario = api.get_user('juan_qenk')
print('Nombre de Usuario: ', usuario.screen_name)
print('Fecha de creación:', usuario.created_at)
print('Descripción:', usuario.description)
print('Número de seguidores:', usuario.followers_count)
print('Localización:', usuario.location)

Nombre de Usuario:  juan_qenk
Fecha de creación: 2012-04-20 16:06:21
Descripción: Biólogo capaz de hacer la fotosíntesis incluso en días nublados. Habla más de 100 idiomas, todos inventados por él en una sola noche.
Número de seguidores: 921
Localización: Agost


Twitter tiene una normativa que regula la frecuencia máxima de peticiones, así como la cantidad máxima de tweets que se pueden extraer, [rate limiting](https://developer.twitter.com/en/docs/basics/rate-limits). Es importante cumplir estos límites para evitar ser bloqueado.

Para el trabajo de análisis que se quiere realizar, es conveniente recuperar tantos tweets como sea posible, al menos unos 1000 por búsqueda. Para ello podemos usar el objeto `cursor` de Tweepy. Este objeto nos permite paginar los resultados ajustandonos a un número de tuits máximo que podemos elegir. 

In [16]:
query = 'biotecnología -filter:retweets'
max_tweets = 1000
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]

Acabamos de generar una lista con todos los tuits que coinciden con la palabra *biotechnology*. Hemos ajustado el número máximo de tuits en 1000, con lo que, si hemos cumplido todos los requisitos que exige la API, 
```python 
len(searched_tweets) == 1000
```


Esta lista de tuits no contiene solo el texto del tuit, es un objeto complejo que tiene mucha información. Si tomamos el primer tuit de la lista, `searched_tweets[0]`, y lo imprimimos en pantalla, podemos ver que toda la información se encuentra en formato json. Json es un formato de texto ligero para el intercambio de datos muy común en internet. 

In [17]:
searched_tweets[0]

Status(_api=<tweepy.api.API object at 0x7f54f0599048>, _json={'created_at': 'Wed Dec 27 13:32:02 +0000 2017', 'id': 946010785851068416, 'id_str': '946010785851068416', 'text': '3P Bio, "actor clave" dentro de la patronal española de Bioempresas https://t.co/GxvCnngz16 #3PBio #biotecnologia… https://t.co/SgOHlhjysJ', 'truncated': True, 'entities': {'hashtags': [{'text': '3PBio', 'indices': [92, 98]}, {'text': 'biotecnologia', 'indices': [99, 113]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/GxvCnngz16', 'expanded_url': 'http://navarracapital.es/3p-bio-actor-clave-dentro-de-la-patronal-espanola-de-bioempresas/', 'display_url': 'navarracapital.es/3p-bio-actor-c…', 'indices': [68, 91]}, {'url': 'https://t.co/SgOHlhjysJ', 'expanded_url': 'https://twitter.com/i/web/status/946010785851068416', 'display_url': 'twitter.com/i/web/status/9…', 'indices': [115, 138]}]}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://coschedule.c

La salida del último comando puede resultar dificilmente legible. Sin embargo podemos leer el contenido de un modo menos doloroso si usamos la función `pprint()`.

In [18]:
import json
import pprint
pprint.pprint(searched_tweets[0]._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Dec 27 13:32:02 +0000 2017',
 'entities': {'hashtags': [{'indices': [92, 98], 'text': '3PBio'},
                           {'indices': [99, 113], 'text': 'biotecnologia'}],
              'symbols': [],
              'urls': [{'display_url': 'navarracapital.es/3p-bio-actor-c…',
                        'expanded_url': 'http://navarracapital.es/3p-bio-actor-clave-dentro-de-la-patronal-espanola-de-bioempresas/',
                        'indices': [68, 91],
                        'url': 'https://t.co/GxvCnngz16'},
                       {'display_url': 'twitter.com/i/web/status/9…',
                        'expanded_url': 'https://twitter.com/i/web/status/946010785851068416',
                        'indices': [115, 138],
                        'url': 'https://t.co/SgOHlhjysJ'}],
              'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 946010785851068416,
 'id_str': '946010785851068

Podemos acceder a un gran número de información tanto del tuit como del usuario que lo creó. 


In [19]:
print('Fecha: ', searched_tweets[0].created_at)
print('Usuario: @' + searched_tweets[0].user.screen_name)
print('Nombre: ', searched_tweets[0].user.name)
print('Descripción: ', searched_tweets[0].user.description)
print('Localización: ', searched_tweets[0].user.location)
print('Verificado: ', searched_tweets[0].user.verified)
print('Seguidores: ', searched_tweets[0].user.followers_count)
print('Texto: ', searched_tweets[0].text)
print('RT: ', searched_tweets[0].retweet_count)
print('Favs: ', searched_tweets[0].favorite_count)

Fecha:  2017-12-27 13:32:02
Usuario: @NavarraCapital
Nombre:  Navarra Capital
Descripción:  Twitter oficial de Navarra Capital, Gabinete de Relaciones Públicas Empresarial de Navarra. Contacto: redaccion@navarracapital.es
Localización:  Pamplona
Verificado:  False
Seguidores:  3542
Texto:  3P Bio, "actor clave" dentro de la patronal española de Bioempresas https://t.co/GxvCnngz16 #3PBio #biotecnologia… https://t.co/SgOHlhjysJ
RT:  0
Favs:  0


In [20]:
columns = ['Usuario','Nombre', 'Fecha', 'Tuit', 'RT', 'Fav', 'Verificado']
df = pd.DataFrame(columns=columns)
for tuit in searched_tweets:
    Fecha = tuit.created_at
    Usuario = tuit.user.screen_name
    Nombre = tuit.user.name
    Verificado= tuit.user.verified
    Texto= tuit.text
    RT= tuit.retweet_count
    Favs= tuit.favorite_count
    df.loc[len(df)]=[Usuario, Nombre, Fecha, Texto, RT, Favs, Verificado ] 
    

In [21]:
df.head()

,Usuario,Nombre,Fecha,Tuit,RT,Fav,Verificado
0,NavarraCapital,Navarra Capital,2017-12-27 13:32:02,"3P Bio, ""actor clave"" dentro de la patronal es...",0,0,False
1,CangasPilar,Pilar Cangas Mtnez,2017-12-27 13:31:49,¿Conoce los principales resultados de la biote...,0,0,False
2,TeleDiagnosis1,TeleDiagnosis,2017-12-27 13:30:58,"En el evento realizado en la ciudad, #Clúster ...",0,0,False
3,fundacionantama,Fundación Antama,2017-12-27 13:10:07,#LoMásVisto2017 Beneficios del maíz Bt en Espa...,1,0,False
4,juanrabio,Juan Ramón Rodríguez,2017-12-27 13:07:19,"2,5 millones para impulsar la biotecnología a...",0,0,False


In [22]:
df.to_pickle('data/biotecnologia.pkl')

Siguiendo el sistema anterior se realizaron las siguientes busquedas:
   
- vacunas ("Genetically modified" OR GMO OR transgenic) AND (vaccine OR vaccination) lang:en exclude:retweets
- medicinas ("Genetically modified" OR GMO OR transgenic) AND (medicina OR drug) lang:en exclude:retweets
- alimentos funcionales ("Genetically modified" OR GMO OR transgenic) AND ("functional food") lang:en exclude:retweets
- arroz dorado ("Genetically modified" OR GMO OR transgenic) AND ("golden rice") lang:en exclude:retweets
- cultivos ("Genetically modified" OR GMO OR transgenic) AND (crops) lang:en exclude:retweets
- pesticidas ("Genetically modified" OR GMO OR transgenic) AND (pesticides) lang:en exclude:retweets
- medio ambiente ("Genetically modified" OR GMO OR transgenic) AND (environment) lang:en exclude:retweets
- bioremediación ("Genetically modified" OR GMO OR transgenic) AND (bioremediation) lang:en exclude:retweets
- cancer ("Genetically modified" OR GMO OR transgenic) AND (cancer) lang:en exclude:retweets
- alergias ("Genetically modified" OR GMO OR transgenic) AND (allergy OR allergies) lang:en exclude:retweets
- Tranferencia de genes ("Genetically modified" OR GMO OR transgenic) AND ("gene transfer") lang:en exclude:retweets
- Desigualdad ("Genetically modified" OR GMO OR transgenic) AND (inequality) lang:en exclude:retweets
- Sociedad ("Genetically modified" OR GMO OR transgenic) AND (society) lang:en exclude:retweets
- Biodiversidad ("Genetically modified" OR GMO OR transgenic) AND (biodiversity) lang:en exclude:retweets
   
Además, usando los datos en directo, se han filtrado los siguientes tuits:   

    ("Genetically modified" OR GMO OR transgenic) lang:en exclude:retweets